In [1]:
# import warnings
# warnings.filterwarnings('ignore')

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # загружаем open_ai_key

In [2]:
import json

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import (
SystemMessage, 
HumanMessage, 
AIMessage,
HumanMessagePromptTemplate, 
SystemMessagePromptTemplate
)
from langchain.chains import ConversationChain, LLMChain
from langchain.memory import ConversationBufferMemory, ChatMessageHistory


In [4]:
model='gpt-3.5-turbo' # 4k токенов контекст
# model='gpt-3.5-turbo-16k' # 16k токенов контекст, подороже
# model='gpt-4' # дорогая для использования

In [5]:
llm = ChatOpenAI(temperature=0.0,model=model)
# chat

## Общее описание структуры

Components of a Startup Idea:

A startup idea consists of three main parts: problem, solution, and insight.
Founders need to construct a compelling hypothesis about their idea's growth potential.
Investors want evidence that the company can grow quickly.
The Problem:

The problem is the initial condition for the startup to grow.
Good problems are popular, growing, urgent, expensive, mandatory, and frequent.
Frequency is crucial, allowing opportunities for conversion.
BJ Fogg's formula for behavior change: motivation, ability, and trigger.
Startups need to address triggers to gain user engagement and retention.
Ideal Startup Problems:

Ideal problems have millions of users, grow rapidly, and are urgent.
Expensive problems with billion-dollar markets are attractive.
Regulatory changes create opportunities.
Frequent problems yield more chances for user conversion.
The Solution:

Avoid "Solution In Search of a Problem" (SISP) approach.
Don't start with technology; start with a problem to solve.
Find problems people have and tailor solutions to them.
The insight explains why the chosen solution will work.
The Insight:

The insight is the company's unfair advantage.
An insight explains why the startup will grow faster than competitors.
Insights can be about the problem, solution, growth channel, or revenue.
Multiple insights can provide a stronger position for success.
Startup Idea as a Hypothesis:

A startup idea is a hypothesis about rapid growth.
Founders must prove this hypothesis true.
Components of the idea (problem, solution, insight) form a compelling story.
The story inspires belief from others, including investors.
Confidence and Iteration:

Confidence in the idea is essential.
Success comes from iterating on the solution, insight, growth channel, and revenue.
Confidence and continuous iteration lead to a successful company.

## V1

Сначала мы задаем первичную инструкцию для чатагпт, в которой описываем его роль.

In [6]:
role = '''Ты превосходный специалист по созданию питч деков. Сейчас ты помогаешь владельцу стартапа написать отличный питч дек.
'''

Затем мы даем контекст инструкцию для чатгпт.
В ней мы пишем следующее:

In [7]:
context = '''Тебе нужно проработать "проблему", которую пытается решить стартап.
'''

А затем детали, которые описывают то, из чего состоит проблема, какую информацию нужно решить.

In [8]:
details = '''
Предпосылки для роста: Описание ситуации, которая позволяет компании быстро расти.

Характеристики хорошей проблемы:

Популярность: Проблема актуальна для большого числа людей.
Рост: Рынок с этой проблемой растет быстрее других.
Срочность: Проблему нужно решить немедленно.
Затраты: Решение проблемы дорого, что может подразумевать высокую оплату.
Обязательность: Проблема требует обязательного решения.
Частота: Проблема возникает часто, люди сталкиваются с ней регулярно.
Связь с частотой: Частота встречаемости проблемы важна, так как частые взаимодействия предоставляют больше возможностей для конверсии.

Инсайт в проблему: Поиск уникального взгляда на проблему, который объясняет, почему ваше решение сможет расти быстро.

Идеальные характеристики проблемы для стартапа:

Миллионы пользователей сталкиваются с этой проблемой.
Рынок с проблемой растет на 20% ежегодно.
Проблему пытаются решить в данный момент.
Решение проблемы потребует больших затрат, что открывает возможности для высокой оплаты.
Законодательство изменилось, создавая дополнительные проблемы для решения.
Проблему необходимо решать 
несколько раз в день.
Суть:
Проблема – это начальные условия, определяющие контекст, в котором ваша компан
ия может расти быстро. Хорошая проблема популярна, растет, требует срочного решения, дорогостояща, обязательн
а и встречается часто. Инсайт в проблему показывает, почему ваше решение имеет конкурентное преим
ущество в плане роста. Выбирайте проблемы, которые соответствуют этим характе инвесторовм,  и привлечь внимание пользователей.'''

И инструкцию, в которой просим чатгпт собрать необходимое кол-во информации, чтобы выяснить какова проблема  
в соответствии с деталями выше.

In [9]:
instruction = '''Твоя задача собрать информацию у владельца стартапа о проблеме, которую они пытаются решить.
И сделать это в таком виде, в котором были бы ясно видны и понятны те характеристики, которые делают проблему "хорошей" для инвестора.
'''

Здесь мы просто оставляем место для истории разговора и места для нового сообщения.

In [10]:
# tech_instructions = '''
# Текущий разговор:
# {history}
# Human: {text}
# AI:'''

In [11]:
tech_instructions = '''
Текущий разговор:
{history}'''

In [12]:
conversation_prompt = role+context+details+instruction+tech_instructions

In [13]:
# conversation_prompt

Создаем память.

In [14]:
memory = ConversationBufferMemory(
    return_messages=False
)

Создаем шаблон, в который помещаем системное сообщение, поле для ввода текста пользователя и пустое поле ассистента.

In [15]:
template = ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate.from_template(conversation_prompt),
     HumanMessagePromptTemplate.from_template("{input}"),
     AIMessage(content='')
    
    ]
)

In [16]:
# template.format_messages(history='ИСТОРИЯ',input='ВВОД')

Создаем саму цепь, к которой будем обращаться в разговоре.

In [18]:
conversation = ConversationChain(
    prompt=template,
    llm=llm, 
    memory=memory,
    verbose=True
)

Теперь напишем промпт для первого сообщения, в котором чатгпт прояснит контекст и то, как он будет выяснять детали.

In [19]:
first_human_message_as_instruction = '''Привет! Я владелец стартапа. Объясни, что ты хочешь у меня узнать и в каком ключе ты хочешь получаться ответы. И попроси меня описать проблему, которую пытается решить мой стартап, настолько полно и подробно, насколько возможно.'''

In [83]:
conversation(first_human_message_as_instruction)



> Entering new ConversationChain chain...
Prompt after formatting:
System: 
Ты превосходный специалист по созданию питч деков. Сейчас ты помогаешь владельцу стартапа написать отличный питч дек.
Тебе нужно проработать "проблему", которую пытается решить стартап.

Предпосылки для роста: Описание ситуации, которая позволяет компании быстро расти.

Характеристики хорошей проблемы:

Популярность: Проблема актуальна для большого числа людей.
Рост: Рынок с этой проблемой растет быстрее других.
Срочность: Проблему нужно решить немедленно.
Затраты: Решение проблемы дорого, что может подразумевать высокую оплату.
Обязательность: Проблема требует обязательного решения.
Частота: Проблема возникает часто, люди сталкиваются с ней регулярно.
Связь с частотой: Частота встречаемости проблемы важна, так как частые взаимодействия предоставляют больше возможностей для конверсии.

Инсайт в проблему: Поиск уникального взгляда на проблему, который объясняет, почему ваше решение сможет расти быстро.

Идеаль

{'input': 'Привет! Я владелец стартапа. Объясни, что ты хочешь у меня узнать и в каком ключе ты хочешь получаться ответы. И попроси меня описать проблему, которую пытается решить мой стартап, настолько полно и подробно, насколько возможно.',
 'history': '',
 'response': 'Привет! Я специалист по созданию питч деков и моя задача - помочь вам описать проблему, которую ваш стартап пытается решить, таким образом, чтобы она соответствовала характеристикам "хорошей" проблемы для инвестора. Я хотел бы узнать больше о вашей компании и проблеме, которую вы пытаетесь решить, чтобы я мог включить все необходимые детали в питч дек. \n\nМожете ли вы описать проблему, которую ваш стартап решает, настолько подробно, насколько возможно? Пожалуйста, укажите популярность проблемы, рост рынка, срочность решения, затраты, обязательность, частоту и связь с частотой. Также, если у вас есть какие-либо инсайты в проблему, которые объясняют, почему ваше решение сможет расти быстро, пожалуйста, поделитесь ими со

Можем посмотреть последнее сообщение.

In [91]:
memory.chat_memory.messages[-1].content

'Привет! Я специалист по созданию питч деков и моя задача - помочь вам описать проблему, которую ваш стартап пытается решить, таким образом, чтобы она соответствовала характеристикам "хорошей" проблемы для инвестора. Я хотел бы узнать больше о вашей компании и проблеме, которую вы пытаетесь решить, чтобы я мог включить все необходимые детали в питч дек. \n\nМожете ли вы описать проблему, которую ваш стартап решает, настолько подробно, насколько возможно? Пожалуйста, укажите популярность проблемы, рост рынка, срочность решения, затраты, обязательность, частоту и связь с частотой. Также, если у вас есть какие-либо инсайты в проблему, которые объясняют, почему ваше решение сможет расти быстро, пожалуйста, поделитесь ими со мной.'

Приветственный промпт будет генерироваться автоматически, или нет. Решим позже.

Далее стартапер пишет ответ. ЧатГПТ отвечает не сразу.

1. Заранее мы просим чатгпт сгенерировать список вопросов или задаем этот список самостоятельно заранее (этот список фактически проходится по ключевым моментам проблемы, а именно тем характеристиках проблемы, которые привлекательны инвестору)
2. ЧатГПТ берет первый ответ стартапера и пытается на основе только него ответить на этот список вопросов
3. Затем по каждому вопросу и своему ответу он обращается к стартаперу, дает контекст (что он сам попытался ответить на эти вопросы и почему они важны) и при необходимости они со стартапером уточняют ответ

Определим вопросы, на которые чатгпт должен постараться дать себе ответы самостоятельно.

In [20]:
mandatory_questions = '''
<<Вопросы, на которые нужно дать ответ>>

Какие начальные условия существуют, которые позволяют вашей компании расти быстро?
Какие признаки делают эту проблему актуальной и важной для большого числа людей?
Какие данные подтверждают, что рынок с этой проблемой растет быстрее, чем другие рынки?
Почему эта проблема требует срочного решения? Есть ли какие-то события или обстоятельства, усиливающие срочность?
Чем именно дорогостояще является решение этой проблемы? Какие затраты пользователи готовы понести?
В чем заключается обязательность решения данной проблемы? Какие факторы заставляют людей обратить на нее внимание?
Как часто пользователи сталкиваются с этой проблемой? Есть ли у них ежедневные или регулярные потребности, связанные с ней?
'''

# mandatory_questions_list = ['Какие начальные условия существуют, которые позволяют вашей компании расти быстро?',
#                             'Какие признаки делают эту проблему актуальной и важной для большого числа людей?',
#                             'Какие данные подтверждают, что рынок с этой проблемой растет быстрее, чем другие рынки?',
#                             'Почему эта проблема требует срочного решения? Есть ли какие-то события или обстоятельства, усиливающие срочность?',
#                             'Чем именно дорогостояще является решение этой проблемы? Какие затраты пользователи готовы понести?',
#                             'В чем заключается обязательность решения данной проблемы? Какие факторы заставляют людей обратить на нее внимание?',
#                             'Как часто пользователи сталкиваются с этой проблемой? Есть ли у них ежедневные или регулярные потребности, связанные с ней?'
#                            ]
                            

Теперь зададим инструкцию для цепочки: 
* Возьми первый вопрос стартапера
* Постарайся с его помощью ответить на эти вопросы
* В ответах постарайся выделить те вещи, которые делают стартап интересным для инвестора

In [21]:
question_instuctions = '''Твоя задача используя текст попытаться ответить на вопросы.
В ответах сделай акцент на тех вещах, которые повысят интерес инвестора к стартапу.
'''

Добавим описание того, что интересно клиенту. Контект. Возьмем его из созданного ранее.

In [22]:
details = '''
<<Контекст>>
Предпосылки для роста: Описание ситуации, которая позволяет компании быстро расти.

Характеристики хорошей проблемы:

Популярность: Проблема актуальна для большого числа людей.
Рост: Рынок с этой проблемой растет быстрее других.
Срочность: Проблему нужно решить немедленно.
Затраты: Решение проблемы дорого, что может подразумевать высокую оплату.
Обязательность: Проблема требует обязательного решения.
Частота: Проблема возникает часто, люди сталкиваются с ней регулярно.
Связь с частотой: Частота встречаемости проблемы важна, так как частые взаимодействия предоставляют больше возможностей для конверсии.

Инсайт в проблему: Поиск уникального взгляда на проблему, который объясняет, почему ваше решение сможет расти быстро.

Идеальные характеристики проблемы для стартапа:

Миллионы пользователей сталкиваются с этой проблемой.
Рынок с проблемой растет на 20% ежегодно.
Проблему пытаются решить в данный момент.
Решение проблемы потребует больших затрат, что открывает возможности для высокой оплаты.
Законодательство изменилось, создавая дополнительные проблемы для решения.
Проблему необходимо решать 
несколько раз в день.
Суть:
Проблема – это начальные условия, определяющие контекст, в котором ваша компан
ия может расти быстро. Хорошая проблема популярна, растет, требует срочного решения, дорогостояща, обязательн
а и встречается часто. Инсайт в проблему показывает, почему ваше решение имеет конкурентное преим
ущество в плане роста. Выбирайте проблемы, которые соответствуют этим характе инвесторовм,  и привлечь внимание пользователей.'''

И добавим инструкцию, в которой опишем в каком виде хотим получить вывод.

In [59]:
json_output_instruction = '''<<Вывод>>
Ответ выдай в виде JSON.
Ключом будет номер вопроса, а значением вложенный список с вопросом и ответом.

Например
```json
"1":["Какие начальные условия существуют, которые позволяют вашей компании расти быстро?","Твой ответ"],
"2":["Какие признаки делают эту проблему актуальной и важной для большого числа людей?","Твой ответ"],

```
'''

Теперь объединим промпт в один.

In [61]:
questions_prompt_template = question_instuctions \
    + details + mandatory_questions \
    + '<<Текст, с помощью которого нужно ответить на вопросы>>\n{text}' \
    + json_output_instruction

In [62]:
test_description = '''У нас есть телеграм-бот, через который можно откликаться на разовые поручения или заявки на временную работу студенты регистрируются там и находятся в ожидании приходящих заявок мы заранее проговариваем с муниципальным предприятием условия гос.закупки на право заключение договора с обязательством предоставления в  течение 12 часов заявленного количества сотрудников, на некоторое количество раз в сезон в день ИКС уходят оповещения по нашей базе самозанятых о возможности выйти на двухдневную работу.
Для того, чтобы студент имел нужные знания, существуют стажировки. На обучение выпускника тратят определенную сумму денег и не факт, что он останется работать. Мы предлагаем, чтобы студент шел к нам, мы его обучали(давали узконаправленные знания для быстрого трудоустройства), после чего отправляем его на работу на определенное время, где он получает опыт. Спустя 3 месяца мы имем хорошего начального спкциалиста,  который спокойно сможет устроиться на работу. С каждой оплаты работу студента мы забираем 10 процентов себе. Мы планируем пойти дальше и получить лицензию на образовательную деятельность, чтобы иметь возможность разрабатывать свои обучающие программы, проработанные с экспертами предприятий (металлургическими комбинатами, операторами связи, заводами, инженерными компаниями), благодаря которым молодой специалист будет проходить "курс молодого бойца" исходя из реальных потребностей производственного сектора. Образование будет платным и укладывается в рамки ожиданий опрошенных нами респондентов.

Наши преимущества - мы сами студенты среднего профессионального учреждения, мы внутри проблемы и мы целевая аудитория.''' 

In [63]:
questions_prompt = ChatPromptTemplate.from_template(questions_prompt_template)

# Chain 1
questions_chain = LLMChain(llm=llm, prompt=questions_prompt)

test_request = questions_chain.run(test_description)

In [64]:
print(test_request)

{
  "1": ["Какие начальные условия существуют, которые позволяют вашей компании расти быстро?", "Наличие телеграм-бота, который позволяет студентам быстро находить временную работу и откликаться на разовые поручения. Мы также имеем соглашение с муниципальным предприятием о предоставлении сотрудников в течение 12 часов после заявки."],
  "2": ["Какие признаки делают эту проблему актуальной и важной для большого числа людей?", "Проблема актуальна для множества студентов, которые ищут временную работу. Также, с ростом числа самозанятых людей, наш сервис предоставляет им возможность получать дополнительный заработок."],
  "3": ["Какие данные подтверждают, что рынок с этой проблемой растет быстрее, чем другие рынки?", "Мы видим, что количество студентов, ищущих временную работу, постоянно растет. Также, рынок самозанятых людей также растет, что создает дополнительный спрос на наш сервис."],
  "4": ["Почему эта проблема требует срочного решения? Есть ли какие-то события или обстоятельства, у

In [ ]:
test_request

In [65]:
response = json.loads(test_request)

In [68]:
response['1'][0]

'Какие начальные условия существуют, которые позволяют вашей компании расти быстро?'

Я пока не понимаю, как потом вывод отдельной цепочки загонять в разговор и как все это обрабатывать. Плюс как контролировать кол-во токенов, чтобы не выйти за лимиты контекста.

## V2

Попытаемся решить всю задачу по созданию слайда или нескольких о проблеме целиком.

Для этого постараемся в системной инструкции описать все, что хотим получить на выходе, все вопросы, которые нужно прояснить со стартапером и общий контекст.

Проходимся, примерно, по тем же этапам написания промпта. Я сделаю заметку там, где он будет отличаться от первой версии.

Сначала мы задаем первичную инструкцию для чатагпт, в которой описываем его роль.

In [25]:
role = '''Ты превосходный специалист по созданию питч деков. Сейчас ты помогаешь владельцу стартапа написать отличный питч дек.
'''

Затем мы даем контекст инструкцию для чатгпт.
В ней мы пишем следующее:

Изменения:
* Уберем контекст вообще, потому что он копирует инструкцию.

In [26]:
# context = '''Тебе нужно проработать "проблему", которую пытается решить стартап.
# '''

А затем детали, которые описывают то, из чего состоит проблема, какую информацию нужно решить.

Здесь нужно внести изменения:
* Добавить вступление о том, что таковы характеристики проблемы, которая привлекательная для инвесторов.
* Дать больше контекста по каждой характеристике.

In [27]:
details = '''
Первым этапом проработки питч-дека является проработка проблемы.
Проблема - это то, что пытается решить стартап. 
Проблема – это начальные условия, определяющие контекст, в котором ваша компания может расти быстро. Хорошая проблема популярна, растет, требует срочного решения, дорогостояща, обязательна и встречается часто. Инсайт в проблему показывает, почему ваше решение имеет конкурентное преимущество в плане роста. Выбирайте проблемы, которые соответствуют этим характеристикам, чтобы убедить инвесторов и привлечь внимание пользователей.
У проблемы, которая привлекательная для инвесторов есть характеристики:

1. Популярность: Популярность проблемы означает, что она актуальна для большого числа людей. Большой спрос на решение этой проблемы подразумевает, что существует значительная группа людей, которые сталкиваются с ней и ищут способы ее решения. Чем более широко проблема распространена, тем выше потенциал для быстрого роста вашей компании. В сущности, это уже текущий объем потенциальных клиентов.

2. Рост: Рост проблемы означает, что рынок, связанный с этой проблемой, растет быстрее, чем другие рынки. Это может быть обусловлено повышенным спросом на решение данной проблемы или изменениями в обществе, экономике или технологиях, которые стимулируют увеличение количества людей, сталкивающихся с этой проблемой. Иметь дело с растущей проблемой означает, что у вашей компании есть потенциал привлечь все больше пользователей и клиентов, что может способствовать быстрому росту бизнеса. В сущности, это степень роста рынка этой проблемы.

3. Срочность: Проблема считается срочной, если ее решение требуется немедленно или в ближайшем будущем. Срочные проблемы обычно вызывают дискомфорт, требуют внимания и действий в краткосрочной перспективе. Когда пользователи осознают, что проблема требует немедленного решения, это может способствовать активному поиску подходящих решений, включая продукты и услуги, предлагаемые вашим стартапом. В сущности, это характеристика или степень безотлагательности решения проблемы.

4. Затраты: В данном контексте затраты на решение проблемы скорее означают, что пользователи могут быть готовы платить значительные суммы за эффективное решение данной проблемы. То есть, если проблема дорогостояща для пользователей, то ваш стартап может предлагать решение, которое оправдывает эти затраты. В сущности, это денежный объем рынка проблемы.

5. Обязательность: Проблема с обязательностью означает, что люди должны решить ее по каким-то обстоятельствам. Это может быть связано с законодательством, регулированием, требованиями индустрии или другими факторами, которые обязывают пользователей найти способы решить данную проблему. Проблемы, которые обязательны к решению, создают своеобразную неотложность для пользователей, и это может стимулировать их активное взаимодействие с вашим стартапом, предоставляющим решение этой обязательной проблемы.

6. Частота: Проблема с частотой означает, что она проявляется у людей с высокой регулярностью или в определенные моменты времени. Это может быть проблема, с которой пользователи сталкиваются неоднократно, возможно, даже ежедневно или еженедельно. Проблемы с высокой частотой обеспечивают множество возможностей для вашего стартапа взаимодействовать с пользователями и предоставлять им решения. Когда проблема проявляется с высокой частотой, это может создать более постоянную потребность в решении, что способствует удержанию пользователей и стимулирует повторные взаимодействия с вашим продуктом или услугой.

Пример идеальных характеристик проблем для стартапа:
Миллионы пользователей сталкиваются с этой проблемой.
Рынок с проблемой растет на 20% ежегодно.
Проблему пытаются решить сразу.
Решение проблемы потребует больших затрат, что открывает возможности для высокой оплаты.
Законодательство изменилось, создавая дополнительные проблемы для решения.
Проблему необходимо решать несколько раз в день.
'''

И инструкцию, в которой просим чатгпт собрать необходимое кол-во информации, чтобы выяснить какова проблема  
в соответствии с деталями выше.
**Здесь нужно модифицировать инструкцию**:  
* Мы добавляем, что на выходе хотим получить содержание одного слайда.
* В этом слайде нужно описать ключевые моменты проблемы
* В создании содержания слайда мы в первую очередь опираемся на то, что говорит стартапер
* Модель, опираясь на слова стартапера, должна постараться написать эти ключевые моменты заинтересовать инвестора
* Характеристики проблемы, который интересны инвестору находятся в `details`
* Если по мнению модели 

Сначала модели нужно выяснить, достаточно ли стартапер предоставил информации, чтобы выяснить обладает ли их проблема характеристиками проблемы, привелекательной для клиента

In [28]:
instruction = '''
Задачи:
1. Попросить у стартапера информацию о проблеме
2. Постарайся расписать проблему, пройдясь по всем пунктам хорошей проблемы, которая заинтересовывает инвесторов.
2. Определить, достаточно ли стартапер предоставил информации, чтобы выяснить, обладает ли проблема, которую они пытаются решить, такими характеристиками, которые привлекательны инвесторам
3. Если информации недостаточно, то последовательно задавать вопросы стартаперу. Один вопрос за другим. Не переходя к следующему, пока не будет получено достаточно информации. При этом стартапером может задавать уточняющие вопросы и попросить помощи. Тебе нужно будет об этом упомянуть
4. Собрав недостающую информацию, дать детальное описание проблемы по каждому пункту характеристики, которая интересна инвестору с учетом предоставленной информации. Говори прямо соответствует ли эта характеристика тем параметрам, которые необходимы, чтобы считать ее привлекательной для инвестора.
5. Попроси у стартапера обратную связь по этому детальному описанию. Если он решит внести коррективы - сделай так.
6. Сформируй содержание слайда с акцентом на те характеристики, которые являются наиболее привлекательными для инвестора.
'''

In [29]:
# instruction = '''Твоя задача собрать информацию у владельца стартапа о проблеме, которую они пытаются решить.
# И сделать это в таком виде, в котором были бы ясно видны и понятны те характеристики, которые делают проблему "хорошей" для инвестора.
# '''

Здесь мы просто оставляем место для истории разговора и места для нового сообщения.

In [30]:
# tech_instructions = '''
# Текущий разговор:
# {history}
# Human: {text}
# AI:'''

In [31]:
tech_instructions = '''
Текущий разговор:
{history}'''

Изменения:
* удалили `context`

In [32]:
conversation_prompt = role+details+instruction+tech_instructions

In [14]:
# conversation_prompt

Создаем память.

In [55]:
memory = ConversationBufferMemory(
    return_messages=False
)

Создаем шаблон, в который помещаем системное сообщение, поле для ввода текста пользователя и пустое поле ассистента.

In [56]:
template = ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate.from_template(conversation_prompt),
     HumanMessagePromptTemplate.from_template("{input}"),
     AIMessage(content='')
    
    ]
)

In [44]:
# template.format_messages(history='ИСТОРИЯ',input='ВВОД')

Создаем саму цепь, к которой будем обращаться в разговоре.

In [63]:
streaming_llm = ChatOpenAI(  
 streaming=True,  
 # callbacks=[StreamingStdOutCallbackHandler()],  
 temperature=0,  
 # openai_api_key=openai_api_key,
 model='gpt-3.5-turbo-16k',
);
conversation = ConversationChain(
    prompt=template,
    llm=streaming_llm, 
    memory=memory,
    verbose=True
)

In [61]:
memory.chat_memory.messages[-1].content

IndexError: list index out of range

In [62]:
conversation = ConversationChain(
    prompt=template,
    llm=llm, 
    memory=memory,
    verbose=True
)

Теперь напишем промпт для первого сообщения, в котором чатгпт прояснит контекст и то, как он будет выяснять детали.

In [64]:
first_human_message_as_instruction = '''Привет! Я владелец стартапа. Объясни, что ты хочешь у меня узнать и в каком ключе ты хочешь получаться ответы. И попроси меня описать проблему, которую пытается решить мой стартап, настолько полно и подробно, насколько возможно.'''

In [65]:
conversation(first_human_message_as_instruction)



> Entering new ConversationChain chain...
Prompt after formatting:
System: Ты превосходный специалист по созданию питч деков. Сейчас ты помогаешь владельцу стартапа написать отличный питч дек.

Первым этапом проработки питч-дека является проработка проблемы.
Проблема - это то, что пытается решить стартап. 
Проблема – это начальные условия, определяющие контекст, в котором ваша компания может расти быстро. Хорошая проблема популярна, растет, требует срочного решения, дорогостояща, обязательна и встречается часто. Инсайт в проблему показывает, почему ваше решение имеет конкурентное преимущество в плане роста. Выбирайте проблемы, которые соответствуют этим характеристикам, чтобы убедить инвесторов и привлечь внимание пользователей.
У проблемы, которая привлекательная для инвесторов есть характеристики:

1. Популярность: Популярность проблемы означает, что она актуальна для большого числа людей. Большой спрос на решение этой проблемы подразумевает, что существует значительная группа люде

{'input': 'Привет! Я владелец стартапа. Объясни, что ты хочешь у меня узнать и в каком ключе ты хочешь получаться ответы. И попроси меня описать проблему, которую пытается решить мой стартап, настолько полно и подробно, насколько возможно.',
 'history': '',
 'response': 'Привет! Я специалист по созданию питч деков и моя задача - помочь вам создать отличный питч дек для вашего стартапа. Чтобы начать, мне нужно получить от вас информацию о проблеме, которую ваш стартап пытается решить. Я хотел бы получить максимально полное и подробное описание этой проблемы, чтобы я мог проработать ее характеристики и определить, соответствует ли она требованиям, которые привлекательны для инвесторов. Пожалуйста, опишите проблему вашего стартапа настолько подробно, насколько возможно, и укажите, какие именно характеристики этой проблемы вы считаете наиболее важными. Если у вас возникнут вопросы или потребуется уточнение, не стесняйтесь задавать их.'}

In [66]:
test_description = '''У нас есть телеграм-бот, через который можно откликаться на разовые поручения или заявки на временную работу студенты регистрируются там и находятся в ожидании приходящих заявок мы заранее проговариваем с муниципальным предприятием условия гос.закупки на право заключение договора с обязательством предоставления в  течение 12 часов заявленного количества сотрудников, на некоторое количество раз в сезон в день ИКС уходят оповещения по нашей базе самозанятых о возможности выйти на двухдневную работу.
Для того, чтобы студент имел нужные знания, существуют стажировки. На обучение выпускника тратят определенную сумму денег и не факт, что он останется работать. Мы предлагаем, чтобы студент шел к нам, мы его обучали(давали узконаправленные знания для быстрого трудоустройства), после чего отправляем его на работу на определенное время, где он получает опыт. Спустя 3 месяца мы имем хорошего начального спкциалиста,  который спокойно сможет устроиться на работу. С каждой оплаты работу студента мы забираем 10 процентов себе. Мы планируем пойти дальше и получить лицензию на образовательную деятельность, чтобы иметь возможность разрабатывать свои обучающие программы, проработанные с экспертами предприятий (металлургическими комбинатами, операторами связи, заводами, инженерными компаниями), благодаря которым молодой специалист будет проходить "курс молодого бойца" исходя из реальных потребностей производственного сектора. Образование будет платным и укладывается в рамки ожиданий опрошенных нами респондентов.

Наши преимущества - мы сами студенты среднего профессионального учреждения, мы внутри проблемы и мы целевая аудитория.''' 

In [67]:
conversation(test_description)



> Entering new ConversationChain chain...
Prompt after formatting:
System: Ты превосходный специалист по созданию питч деков. Сейчас ты помогаешь владельцу стартапа написать отличный питч дек.

Первым этапом проработки питч-дека является проработка проблемы.
Проблема - это то, что пытается решить стартап. 
Проблема – это начальные условия, определяющие контекст, в котором ваша компания может расти быстро. Хорошая проблема популярна, растет, требует срочного решения, дорогостояща, обязательна и встречается часто. Инсайт в проблему показывает, почему ваше решение имеет конкурентное преимущество в плане роста. Выбирайте проблемы, которые соответствуют этим характеристикам, чтобы убедить инвесторов и привлечь внимание пользователей.
У проблемы, которая привлекательная для инвесторов есть характеристики:

1. Популярность: Популярность проблемы означает, что она актуальна для большого числа людей. Большой спрос на решение этой проблемы подразумевает, что существует значительная группа люде

{'input': 'У нас есть телеграм-бот, через который можно откликаться на разовые поручения или заявки на временную работу студенты регистрируются там и находятся в ожидании приходящих заявок мы заранее проговариваем с муниципальным предприятием условия гос.закупки на право заключение договора с обязательством предоставления в  течение 12 часов заявленного количества сотрудников, на некоторое количество раз в сезон в день ИКС уходят оповещения по нашей базе самозанятых о возможности выйти на двухдневную работу.\nДля того, чтобы студент имел нужные знания, существуют стажировки. На обучение выпускника тратят определенную сумму денег и не факт, что он останется работать. Мы предлагаем, чтобы студент шел к нам, мы его обучали(давали узконаправленные знания для быстрого трудоустройства), после чего отправляем его на работу на определенное время, где он получает опыт. Спустя 3 месяца мы имем хорошего начального спкциалиста,  который спокойно сможет устроиться на работу. С каждой оплаты работу 

In [68]:
print(memory.chat_memory.messages[-1].content)

Спасибо за предоставленную информацию о проблеме, которую ваш стартап пытается решить. Опишу проблему, исходя из предоставленных данных, и проработаю ее характеристики, которые могут быть привлекательными для инвесторов.

Проблема: Недостаток временных работников для выполнения разовых поручений и заявок.

Характеристики проблемы:

1. Популярность: Проблема наблюдается у множества компаний и организаций, которые нуждаются во временных работниках для выполнения разовых поручений или заявок. Это означает, что существует значительный спрос на решение этой проблемы.

2. Рост: Рынок временных работников растет, так как все больше компаний и организаций ищут способы найма временных сотрудников для выполнения разовых поручений. Это может быть обусловлено изменениями в экономике или технологиях, которые стимулируют увеличение количества компаний, нуждающихся во временных работниках.

3. Срочность: Проблема требует немедленного решения, так как компании и организации нуждаются во временных рабо